### 대이터를 효과적으로 전달하는 방법

- `RunnablePassthrough()`: 입력을 변경하지 않거나 추가 키를 더하여 전달하는 방법입니다. 막약 `RunnablePassthrough()`가 단독으로 호출되면 단순히 입력을 받아 그대로 다음 단계에 전달합니다.
- `RunnablePassthrough.assign(...)`: 입력을 받아 assign 함수에 전달된 추가 인수를 추가합니다.

### RunnablePassthrough

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("다음 문장을 영어로 번역하세요: {input}")
llm = ChatOpenAI(temperature=0)

chain = prompt | llm

In [3]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke({"input": "안녕하세요"})  # 입력을 그대로 전달

{'input': '안녕하세요'}

아래는 `RunnablePassthrough()` 로 체인을 생성하는 예제입니다.

In [4]:
runnable_chain = {
    "input": RunnablePassthrough()
} | chain  # RunnablePassthrough() 를 사용해서 invoke() 때 키:값 형식이 아니라 값만 받을 수 있게 하는 문법을 많이 사용한다.
runnable_chain.invoke("안녕하세요")

AIMessage(content='Hello', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 25, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ffb901cc-4446-49ff-ad6e-7d0389c77dfd-0', usage_metadata={'input_tokens': 25, 'output_tokens': 1, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

아래는 `RunnablePassthrough.assign()` 로 체인을 생성하는 예제입니다.

In [6]:
RunnablePassthrough().assign(
    new_input=lambda x: x["input"] + " 좋은 하루 되세요!"
).invoke({"input": "안녕하세요."})

{'input': '안녕하세요.', 'new_input': '안녕하세요. 좋은 하루 되세요!'}

### RunnableParallel

In [7]:
from langchain_core.runnables import RunnableParallel

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough().assign(
        new_input=lambda x: x["input"] + " 좋은 하루 되세요!"
    ),
    modified=lambda x: x["input"] + " 반갑습니다!",
)
runnable.invoke({"input": "안녕하세요."})

{'passed': {'input': '안녕하세요.'},
 'extra': {'input': '안녕하세요.', 'new_input': '안녕하세요. 좋은 하루 되세요!'},
 'modified': '안녕하세요. 반갑습니다!'}

Chain에 RunnableParallel 를 사용하는 예제입니다.

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

llm = ChatOpenAI(temperature=0)

chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 수도는?")
    | llm
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 면적은?")
    | llm
)

combined_chain = RunnableParallel(capital=chain1, area=chain2)

combined_chain.invoke("Korea")

{'capital': AIMessage(content='서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 14, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e13007cc-2067-4473-95c0-100b387feb75-0', usage_metadata={'input_tokens': 14, 'output_tokens': 5, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'area': AIMessage(content='대한민국의 총 면적은 약 100,363km² 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 15, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens